<a href="https://colab.research.google.com/github/desean1625/scatter_test/blob/main/scatter_plot_track_sorting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from numpy.random import default_rng

total_points = 1000 #Total number of points generated 
limits = [[-10,10],[-10,10]] #x/y limits for data generation
number_of_possible_clusters = 5 
add_random_noise = True #hopefully the model will filter the noise from the clusters
batch_size = 32 
randomize_points = True #randomizes the points array so they aren't sequential
ranomization_level = .1 #percent of the array we ar going to shuffle


total = total_points

In [ ]:
def conv_block(fs, x, activation = 'relu'):
  conv  = layers.Conv2D(fs, (3, 3), padding = 'same', activation = activation)(x)
  bnrm  = layers.BatchNormalization()(conv)
  drop  = layers.Dropout(0.5)(bnrm)
  return drop

def residual_block(fs, x):
  y = conv_block(fs, x)
  y = conv_block(fs, y)
  y = conv_block(fs, y)
  return layers.Concatenate(axis = -1)([x, y])

inp = keras.Input(shape = (total,2))
out = inp
out = layers.Concatenate(axis=1)([out, out,out])
out = layers.Reshape((3,total,2))(out)
out = residual_block(16, out) #create convolutional filters
out  = layers.MaxPooling2D(pool_size = (2, 2))(out)
out = residual_block(16, out) #create convolutional filters
out = layers.Dense(32)(out)
out = layers.Dense(8)(out)
out = layers.Dense(4)(out)
out = layers.Dense(2)(out)
out = layers.Dense(1)(out)
out = layers.Flatten()(out)
out = layers.Dense(total)(out) #prediction layer output for every input point
model = keras.models.Model(inputs = inp, outputs = out)
model.summary()

In [ ]:
#Create training data generator
import math
import random

def rng_swap(a,i):
    rng = random.randint(0,len(a)-1)
    a[i],a[rng] = a[rng],a[i]
def shuffle(a,swap_chance):
    for i in range(len(a)):
        if random.random()<0.1:
           rng_swap(a,i)

def generate_random_path(x,y,length,heading_std=.01,speed_start=.01,speed_std=.01):
    global limits
    points = []
    points.append([x,y])

    heading = random.uniform(0,360)
    speed = speed_start

    for i in range(0,length-1):
        heading = heading + random.gauss(0,heading_std)
        heading = heading % 360
        x1 = points[i][0]+ math.cos(math.radians(heading))*speed
        y1 = points[i][1]+ math.sin(math.radians(heading))*speed
        if x1<limits[0][0]: x1=limits[0][0]
        if x1>limits[0][1]: x1=limits[0][1]
        if y1<limits[1][0]: y1=limits[1][0]
        if y1>limits[1][1]: y1=limits[1][1]
        points.append([x1,y1])
        speed = speed + random.gauss(0,speed_std)
    return points

def norm(xy):
  global limits
  x_range = limits[0]
  y_range = limits[1]
  x = (xy[0]-x_range[0])/(x_range[1]-x_range[0])
  y = (xy[1]-y_range[0])/(y_range[1]-y_range[0])
  return (x,y)
def rand_point(x,y):
  deviation = .01
  xmin = x -(x*deviation)
  xmax = x + (x*deviation)
  ymin = y -(y*deviation)
  ymax = y + (y*deviation)
  x = random.uniform(xmin, xmax)
  y = random.uniform(ymin, ymax)
  return x,y
def create_sample(total,randomize_points=True):
  global limits
  global ranomization_level
  global add_random_noise
  clusters = random.randint(2,6)
  num = int((total)//clusters)
  allx =[]
  ally = []
  for x in range(1,clusters+1):
    n = (random.uniform(*limits[0]),random.uniform(*limits[1]))
    points = generate_random_path(n[0],n[1],num)
    allx.extend(points)
    for i in range(0,num):
      ally.append(x)
    if x == clusters and total-len(allx) != 0:
      #top up the remainder
      points = generate_random_path(allx[len(allx)-1][0],allx[len(allx)-1][1],total-len(allx))
      allx.extend(points)
      for z in range(0,total-len(ally)):
        ally.append(x)
  if add_random_noise:
    rng = default_rng()
    numbers = rng.choice(len(allx), size=int(len(allx)*ranomization_level), replace=False)
    for i in numbers:
      n = (random.uniform(*limits[0]),random.uniform(*limits[1]))
      allx[i] =n
      ally[i] = 0
  normx = list(map(norm,allx))
  x = np.array([np.hstack(t) for t in normx],dtype=np.float32)
  y = np.array(ally,dtype=np.float32)
  if randomize_points:
    z = list(zip(x,y))
    shuffle(z,.1)
    x,y = zip(*z)
  return x,y
def train_gen(total,batch_size=32,randomize_points=True):
  while True:
    x_train = []
    y_train = []
    for x in range(0,batch_size):
      x,y = create_sample(total,randomize_points)
      x_train.append(x)      
      y_train.append(y)
    yield np.array(x_train),np.array(y_train)
gen = train_gen(total,batch_size,randomize_points)
x_train,y_train = next(gen)

In [ ]:
opt = tf.keras.optimizers.Adam(
    learning_rate=0.001
)

def custom_loss(y_true, y_pred):
    # calculating difference between target and predicted values 
    loss = keras.backend.abs(y_true - y_pred)   # (batch_size, 2)
    loss = loss * 1.5          # Make the penalty for being wrong more severe      
    # summing both loss values along batch dimension 
    loss = keras.backend.sum(loss, axis=1)       
    return loss
loss = custom_loss


def total_accuracy(y_true, y_pred):
    equals = (y_true.numpy().flatten() == keras.backend.round(y_pred).numpy().flatten()).sum()  #TODO fix this to use tf functions instead of numpy so we don't need to run_eagerly
    result = equals/len(y_pred.numpy().flatten())
    return result
loss = custom_loss
model.compile(loss=loss, optimizer=opt, metrics=[total_accuracy], run_eagerly=True)

In [ ]:
early_stopping_patience = 15
#early_stopping = tf.keras.callbacks.EarlyStopping(monitor="total_accuracy",patience=early_stopping_patience,restore_best_weights=True)
#model.fit(gen,steps_per_epoch=batch_size, epochs=2000,callbacks=[early_stopping])
model.fit(gen,steps_per_epoch=batch_size, epochs=2000)

In [ ]:
x_train,y_train = next(gen)

r = keras.backend.round(model.predict(x_train))
print("players found",len(np.unique(r.numpy()[0])))
print("players expected",len(np.unique(y_train[0])))
print("predicted",r.numpy()[0][0:10])
print("expected", y_train[0][0:10])
equals = (r.numpy()[0] == y_train[0]).sum()  
result = equals/len(y_train[0])
print("Prediction accuracy",result)
import matplotlib.pyplot as plt
plt.scatter(x_train[0][:, 0],x_train[0][:, 1],c=y_train[0])
plt.show()
plt.scatter(x_train[0][:, 0],x_train[0][:, 1],c=r.numpy()[0])
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(x_train[0][:, 0],x_train[0][:, 1],c=y_train[0])
plt.show()
plt.scatter(x_train[0][:, 0],x_train[0][:, 1],c=r.numpy()[0])
plt.show()

In [ ]:
from numpy.random import default_rng

rng = default_rng()
numbers = rng.choice(20, size=10, replace=False)
print(numbers)